In [1]:
from google.colab import drive, userdata
drive.mount('/content/drive')
ngrok_auth_token = userdata.get('NGROK_AUTH_TOKEN')
ngrok_static_domain = userdata.get('NGROK_STATIC_DOMAIN')

!rm -rf /content/BIG_MOOC
!git clone https://github.com/ippapi/BIG_MOOC.git

!pip install fastapi nest-asyncio pyngrok uvicorn pyspark
from fastapi import FastAPI, Request
import uvicorn
import nest_asyncio
import socket
import threading
import time
import json
import socket
from threading import Thread
import json
import csv

from pyngrok import ngrok
ngrok.set_auth_token(ngrok_auth_token)

import os
os.environ["NGROK_STATIC_DOMAIN"] = ngrok_static_domain
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!cp /content/drive/MyDrive/spark-3.5.1-bin-hadoop3.tgz /content/
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark
import findspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark

Mounted at /content/drive
Cloning into 'BIG_MOOC'...
remote: Enumerating objects: 924, done.
remote: Counting objects: 100% (240/240), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 924 (delta 143), reused 159 (delta 76), pack-reused 684 (from 1)
Receiving objects: 100% (924/924), 233.90 MiB | 29.39 MiB/s, done.
Resolving deltas: 100% (432/432), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.5 MB/s eta 0:00:00


In [3]:
course_mapping = {}
with open("/content/drive/MyDrive/BIG_MOOC/dataset/course_map.csv", mode="r") as file:
    reader = csv.reader(file)
    for row in reader:
        if len(row) >= 2:
            course_mapping[row[0].strip()] = row[1].strip()

user_mapping = {}
with open("/content/drive/MyDrive/BIG_MOOC/dataset/user_map.csv", mode="r") as file:
    reader = csv.reader(file)
    for row in reader:
        if len(row) >= 2:
            user_mapping[row[0].strip()] = row[1].strip()

clients = []

def handle_client(client_socket):
    while True:
        try:
            data = client_socket.recv(1024)
            if not data:
                break
            print("Received:", data.decode('utf-8'))
        except:
            break
    client_socket.close()
    clients.remove(client_socket)

def start_server():
    server_socket = socket.socket()
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server_socket.bind(('localhost', 9999))
    server_socket.listen(5)
    print("Socket server listening on port 9999")

    while True:
        client_socket, addr = server_socket.accept()
        print(f"{client_socket} connected with addr: {addr}")
        clients.append(client_socket)
        Thread(target=handle_client, args=(client_socket,), daemon=True).start()

def send_to_stream_socket(message):
    if isinstance(message, dict):
        message = json.dumps(message)
    data = (message + '\n').encode('utf-8')

    for client in clients[:]:
        try:
            client.sendall(data)
            print("Sent to socket")
        except Exception as e:
            print("Failed to send to socket:", e)
            clients.remove(client)

app = FastAPI()
nest_asyncio.apply()

@app.post("/receive")
async def receive_data(request: Request):
    data = await request.json()
    print("Original data:", data)

    user_id = data.get('result', {}).get('user_id', 0)
    course_id = data.get('result', {}).get('course_id', 0)

    print(f"User ID: {user_id}, Course ID: {course_id}")

    if course_id in course_mapping:
        data["result"]["course_id"] = int(course_mapping[course_id]) + 1
    else:
        data["result"]["course_id"] = 0

    if user_id in user_mapping:
        data["result"]["user_id"] = int(user_mapping[user_id])
    else:
        data["result"]["user_id"] = 0

    print("Mapped data:", data)
    send_to_stream_socket(data)

    return {"status": "received"}

Thread(target=start_server, daemon=True).start()

public_url = ngrok.connect(8000)
print("Public URL:", public_url)

Thread(target=lambda: uvicorn.run(app, host="localhost", port=8000), daemon=True).start()


Exception in thread Thread-14 (start_server):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-c56b75da734e>", line 32, in start_server
OSError: [Errno 98] Address already in use


Public URL: NgrokTunnel: "https://a835-34-83-247-208.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
!python /content/BIG_MOOC/stream_process/main.py

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/10 00:06:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/10 00:06:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/05/10 00:06:30 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
[Main] Launched 2 workers
Started local training with 2 processes
25/05/10 00:06:34 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/05/10 00:06:34 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
25/05/10 00:06:36 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-159d8d92-6300-4f3b-b4a7-ef410b2fcfd3. If it's 

In [ ]:
from pyspark.ml.torch.distributor import TorchDistributor

distributor = TorchDistributor(
    num_processes=2,
    local_mode=True,
    use_gpu=False,
)

# Path to your training script
distributor.run(
    "/content/BIG_MOOC/stream_process/pretrain_model/distributed_main.py",
    "--num_epochs", 50,
    "--sequence_size", 15,
    "--embedding_dims", 64,
    "--mode", "train",
    "--model_version", "_s15_e64_train",
    "--device", "cpu",
)

INFO:TorchDistributor:Started local training with 2 processes



*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
*****************************************
Rank: 0, Local rank: 0, World size: 2
Rank: 1, Local rank: 1, World size: 2
/usr/local/lib/python3.11/dist-packages/torch/utils/data/sampler.py:77: UserWarning: `data_source` argument is not used and will be removed in 2.2.0.You may still have custom implementation that utilizes it.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/sampler.py:77: UserWarning: `data_source` argument is not used and will be removed in 2.2.0.You may still have custom implementation that utilizes it.
  warnings.warn(
Epoch 1:   0%|          | 0/782 [00:00<?, ?batch/s]/content/BIG_MOOC/stream_process/pretrain_model/utils/model.py:58: UserWarning: To copy construct from a tensor, it is recommende

INFO:TorchDistributor:Finished local training with 2 processes


Epoch 50: 100%|██████████| 782/782 [01:28<00:00,  8.82batch/s, loss=0.4847]
Final model saved at /content/drive/MyDrive/BIG_MOOC/train_dir/SASRec_v_s15_e64_train.final.pth


In [ ]:
!python /content/BIG_MOOC/stream_process/pretrain_model/evaluate_distributed.py --state_dict_path "/content/drive/MyDrive/BIG_MOOC/train_dir/SASRec_v_s15_e64_train.final.pth" --device=cpu

Evaluating
/content/BIG_MOOC/stream_process/pretrain_model/utils/model.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  positions_emb = self.position_emb(torch.tensor(filtered_pos, dtype=torch.long).to(self.device))
..................valid (NDCG@10: 0.5319, Hit@10: 0.8184, Recall@10: 0.8184), test (NDCG@10: 0.4939, Hit@10: 0.7793, Recall@10: 0.7793)
Done


In [ ]:
from pyspark.ml.torch.distributor import TorchDistributor

distributor = TorchDistributor(
    num_processes=2,
    local_mode=True,
    use_gpu=False,
)

# Path to your training script
distributor.run(
    "/content/BIG_MOOC/stream_process/pretrain_model/distributed_main.py",
    "--num_epochs", 50,
    "--sequence_size", 15,
    "--embedding_dims", 128,
    "--mode", "train",
    "--model_version", "_s15_e128_train",
    "--device", "cpu",
)

INFO:TorchDistributor:Started local training with 2 processes



*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
*****************************************
Rank: 1, Local rank: 1, World size: 2
Rank: 0, Local rank: 0, World size: 2
/usr/local/lib/python3.11/dist-packages/torch/utils/data/sampler.py:77: UserWarning: `data_source` argument is not used and will be removed in 2.2.0.You may still have custom implementation that utilizes it.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/sampler.py:77: UserWarning: `data_source` argument is not used and will be removed in 2.2.0.You may still have custom implementation that utilizes it.
  warnings.warn(
Epoch 1:   0%|          | 0/782 [00:00<?, ?batch/s]/content/BIG_MOOC/stream_process/pretrain_model/utils/model.py:58: UserWarning: To copy construct from a tensor, it is recommende

INFO:TorchDistributor:Finished local training with 2 processes


In [ ]:
!python /content/BIG_MOOC/stream_process/pretrain_model/evaluate_distributed.py --sequence_size 15 --embedding_dims 128 --state_dict_path "/content/drive/MyDrive/BIG_MOOC/train_dir/SASRec_v_s15_e128_train.final.pth" --device=cpu

Evaluating
/content/BIG_MOOC/stream_process/pretrain_model/utils/model.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  positions_emb = self.position_emb(torch.tensor(filtered_pos, dtype=torch.long).to(self.device))
..................valid (NDCG@10: 0.5549, Hit@10: 0.8351, Recall@10: 0.8351), test (NDCG@10: 0.5183, Hit@10: 0.8027, Recall@10: 0.8027)
Done


In [ ]:
from pyspark.ml.torch.distributor import TorchDistributor

distributor = TorchDistributor(
    num_processes=2,
    local_mode=True,
    use_gpu=False,
)

# Path to your training script
distributor.run(
    "/content/BIG_MOOC/stream_process/pretrain_model/distributed_main.py",
    "--num_epochs", 50,
    "--sequence_size", 20,
    "--embedding_dims", 64,
    "--mode", "train",
    "--model_version", "_s20_e64_train",
    "--device", "cpu",
)

INFO:TorchDistributor:Started local training with 2 processes



*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
*****************************************
Rank: 1, Local rank: 1, World size: 2
Rank: 0, Local rank: 0, World size: 2
/usr/local/lib/python3.11/dist-packages/torch/utils/data/sampler.py:77: UserWarning: `data_source` argument is not used and will be removed in 2.2.0.You may still have custom implementation that utilizes it.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/sampler.py:77: UserWarning: `data_source` argument is not used and will be removed in 2.2.0.You may still have custom implementation that utilizes it.
  warnings.warn(
Epoch 1:   0%|          | 0/782 [00:00<?, ?batch/s]/content/BIG_MOOC/stream_process/pretrain_model/utils/model.py:58: UserWarning: To copy construct from a tensor, it is recommende

INFO:TorchDistributor:Finished local training with 2 processes


In [ ]:
!python /content/BIG_MOOC/stream_process/pretrain_model/evaluate_distributed.py --sequence_size 20 --embedding_dims 64 --state_dict_path "/content/drive/MyDrive/BIG_MOOC/train_dir/SASRec_v_s20_e64_train.final.pth" --device=cpu

Evaluating
/content/BIG_MOOC/stream_process/pretrain_model/utils/model.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  positions_emb = self.position_emb(torch.tensor(filtered_pos, dtype=torch.long).to(self.device))
..................valid (NDCG@10: 0.5277, Hit@10: 0.8120, Recall@10: 0.8120), test (NDCG@10: 0.4904, Hit@10: 0.7712, Recall@10: 0.7712)
Done


In [ ]:
from pyspark.ml.torch.distributor import TorchDistributor

distributor = TorchDistributor(
    num_processes=2,
    local_mode=True,
    use_gpu=False,
)

# Path to your training script
distributor.run(
    "/content/BIG_MOOC/stream_process/pretrain_model/distributed_main.py",
    "--num_epochs", 50,
    "--sequence_size", 20,
    "--embedding_dims", 128,
    "--mode", "train",
    "--model_version", "_s20_e128_train",
    "--device", "cpu",
)

INFO:TorchDistributor:Started local training with 2 processes



*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
*****************************************
Rank: 0, Local rank: 0, World size: 2
Rank: 1, Local rank: 1, World size: 2
/usr/local/lib/python3.11/dist-packages/torch/utils/data/sampler.py:77: UserWarning: `data_source` argument is not used and will be removed in 2.2.0.You may still have custom implementation that utilizes it.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/sampler.py:77: UserWarning: `data_source` argument is not used and will be removed in 2.2.0.You may still have custom implementation that utilizes it.
  warnings.warn(
Epoch 1:   0%|          | 0/782 [00:00<?, ?batch/s]/content/BIG_MOOC/stream_process/pretrain_model/utils/model.py:58: UserWarning: To copy construct from a tensor, it is recommende

INFO:TorchDistributor:Finished local training with 2 processes


In [ ]:
!python /content/BIG_MOOC/stream_process/pretrain_model/evaluate_distributed.py --sequence_size 20 --embedding_dims 128 --state_dict_path "/content/drive/MyDrive/BIG_MOOC/train_dir/SASRec_v_s20_e128_train.final.pth" --device=cpu

Evaluating
/content/BIG_MOOC/stream_process/pretrain_model/utils/model.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  positions_emb = self.position_emb(torch.tensor(filtered_pos, dtype=torch.long).to(self.device))
..................valid (NDCG@10: 0.5507, Hit@10: 0.8291, Recall@10: 0.8291), test (NDCG@10: 0.5147, Hit@10: 0.7941, Recall@10: 0.7941)
Done
